# Prueba residuos en temperatura

Prueba entrenando los modelos en frecuencia con datos normales. La inferencia se realiza con los datos de test correspondientes a pruebas cercanas al fallo. Se pretende observar las diferencias entre las temperaturas estimadas por el modelo y las temperaturas medidas. Estas diferencias o residuos se deben con un cambio en la respuesta del sistema y que pueden estar relacionados con una degradación del mismo. 

## Caracterísiticas y entrenamiento de los modelos
Los modelos son entrenados para estimar la temperatura del estado tomando como entrada el argumento y el módulo de las impedancias complejas para los armónicos -5, 7 y 13. Los modelos entrenados son: SVR, MLP y una regresión lineal como modelo base. Las impedancias complejas utilizadas para el entrenamiento son extraidas de la prueba 21 cuando el motor aún no ha sufrido una degradación del aislamiento aparente. Una vez entrenados los modelos, las temperaturas para las pruebas 24 y 27, donde el motor presenta una degradación del aislamiento en una de sus fases, son estimadas. Cada uno de los modelos son entrenados utilizando hiperparametros establecidos manualmente tras un proceso manual de prueba y error. No se ha realizado ningún tipo "fine tuning" de los hiperparámetros. 

El residuo entre las temperaturas estimadas y las medidas para estas pruebas son estudiados.  

In [1]:
from featureExtraction import featureExtraction
from modelGenerator import modelGenerator
import pandas as pd
import matplotlib.pyplot as plt
from utils import reset_seeds


reset_seeds(seed_value=39)
dataID = "raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27"
modelIDs = ["linearSGD","svr","mlp"]
params = {}

data = featureExtraction(dataID,statorFreqs=[37],testsID=[21,24])  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27
dataTest = featureExtraction(dataID,statorFreqs=[37],testsID=[24,27],scaler_params=data.get_scaler_params())  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27
MODELS = []

for modelID in modelIDs:
    model = modelGenerator(modelID=modelID, data=data,params=params)
    model.train()
    MODELS.append(model)


2023-07-06 14:26:09.137543: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


training <class 'models.linearSGD.linearSGD'>
Test MSE: 1.840042753164543    Test MAE: 1.0562379028109397  
Test MSE: 0.7854663677583116    Test MAE: 0.624865595993937  


/home/diego/.conda/envs/hiwind/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 6)]               0         
                                                                 
 dense_0 (Dense)             (None, 130)               910       
                                                                 
 dense_1 (Dense)             (None, 140)               18340     
                                                                 
 dense (Dense)               (None, 1)                 141       
                                                                 
Total params: 19,391
Trainable params: 19,391
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300


2023-07-06 14:26:24.139829: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 14:26:24.413869: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22309 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


191/397 [=============>................] - ETA: 0s - loss: 13475.6855 - mean_squared_error: 13475.6855 - mean_absolute_error: 112.7283

2023-07-06 14:26:25.056251: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


397/397 [==============================] - 1s 1ms/step - loss: 9887.7139 - mean_squared_error: 9887.7139 - mean_absolute_error: 93.9952 - val_loss: 4370.5981 - val_mean_squared_error: 4370.5981 - val_mean_absolute_error: 61.9165
Epoch 2/300
397/397 [==============================] - 0s 936us/step - loss: 2249.1279 - mean_squared_error: 2249.1279 - mean_absolute_error: 42.5975 - val_loss: 1173.4684 - val_mean_squared_error: 1173.4684 - val_mean_absolute_error: 30.6525
Epoch 3/300
397/397 [==============================] - 0s 913us/step - loss: 509.5661 - mean_squared_error: 509.5661 - mean_absolute_error: 18.5328 - val_loss: 203.7002 - val_mean_squared_error: 203.7002 - val_mean_absolute_error: 11.6190
Epoch 4/300
397/397 [==============================] - 0s 918us/step - loss: 101.4928 - mean_squared_error: 101.4928 - mean_absolute_error: 7.8148 - val_loss: 50.1805 - val_mean_squared_error: 50.1805 - val_mean_absolute_error: 5.6227
Epoch 5/300
397/397 [==============================] -

In [2]:
X_test = dataTest.X
y_test = dataTest.y
Y_test_est = []
for model in MODELS:
    Y_test_est.append(model.predict(X_test))


17/17 [==============================] - 0s 618us/step


In [5]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import numpy as np
output_notebook()

MODEL_LABELS = {"mlp": "Multi layer perceptron","svr": "Support Vector Regression", "linearSGD":"Linear regression"}

for ii,model in enumerate(MODELS):
    p = figure(title=f"Residuals model: {MODEL_LABELS[modelIDs[ii]]}",width=900,height=350)
    p.line(np.arange(len(y_test)),y_test, line_width=2,line_alpha=0.4, line_color='gray', legend_label="Ground truth" )
    p.line(np.arange(len(y_test)),Y_test_est[ii], line_width=2, line_alpha=0.8,line_color='green',legend_label="Estimation")
    p.legend.location = 'bottom_right'
    p.yaxis.axis_label = r"T (ºC)"
    p.xaxis.axis_label = r"sample"
    show(p)



Loading BokehJS ...

In [4]:
from scipy import signal

b = signal.firwin(151,0.01)

for ii,model in enumerate(MODELS):
    residuals = np.square(y_test-Y_test_est[ii])
    residuals_ = signal.filtfilt(b, 1, residuals)
    p = figure(title=f"Abs residuals model: {MODEL_LABELS[modelIDs[ii]]}",width=900,height=250,y_range=(0,5))
    p.line(np.arange(len(y_test)),residuals, line_width=2, line_alpha=0.3,line_color='gray',legend_label="normal")
    p.line(np.arange(len(y_test)),residuals_, line_width=2, line_alpha=0.8,line_color='blue',legend_label="filtered")
    p.legend.location = 'top_right'
    p.yaxis.axis_label = r"$$|T-\hat{T}|^2_2$$"
    p.xaxis.axis_label = r"sample"
    show(p)
    